In [4]:
# Bibliotecas de uso geral
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [7]:
#
#
#  Função que implementa o SFS (Sequential Forward Selection) 
#  para problemas de aprendizagem supervisionada.
#
#  Entradas : X_{m, n} = conjunto de características
#             y_{m, 1} = vetor de rótulos
#             k = parâmetro das melhores k <= n características
#             f = função de avaliação
#             test_size = tamanho do conjunto de testes
#             random_state = parâmetro do train_test_split
#
#
#  Saídas : scores_{k, 1}  = Scores acumulados do 1 até o k 
#                            subconjunto de características
# 
#           indexes_{k, 1} = Índices de 1 até k que formam
#                            subconjuntos ótimos de características
#
def SFS(X, y, k, f, test_size=0.25, random_state=1):
    
    # Guardar os indices e os scores
    indexes  = []
    scores   = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    
    # Indices para serem visitados (lista de busca)
    ind = list(range(X.shape[1]))
    
    for i in range(k):
        
        # Melhor por iteração
        s_best = np.finfo(np.float64).min
        j_best = 0
        
        # Seleção da melhor combinação
        for j in range(len(ind)):      
            xj_train = X_train[:, indexes + [ind[j]]]           
            xj_test  = X_test[:, indexes + [ind[j]]]           
            s_temp = f(xj_train, y_train, xj_test, y_test)
            if s_temp > s_best:
                s_best = s_temp
                j_best = j            
        
        # Adiciona o melhor aos indices
        indexes.append(ind[j_best])
        
        # Remove o melhor encontrado da lista de busca
        del ind[j_best]
        
        # Guarda o score da iteração i
        scores.append(s_best)
        
    return scores, indexes

In [44]:
# Dataset de vinhos
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                             'ml/machine-learning-databases/'
                             'wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol',
                    'Malic acid', 'Ash',
                    'Alcalinity of ash', 'Magnesium',
                    'Total phenols', 'Flavanoids',
                    'Nonflavanoid phenols',
                    'Proanthocyanins',
                    'Color intensity', 'Hue',
                    'OD280/OD315 of diluted wines',
                    'Proline']

# Pegando (X, y)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# Dividindo conjunto de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0, stratify=y)

# Normalizando os dados com zscore
zscore = StandardScaler()
Xz_train = zscore.fit_transform(X_train)
Xz_test  = zscore.transform(X_test)

# Criando classificador para avaliação
clf = LogisticRegression(solver='liblinear', multi_class='ovr')

# Função de avaliação
def f(X_train, y_train, X_test, y_test, clf=clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Selecionando de 1 até 11 melhores
k=11
(scores, indexes) = SFS(Xz_train, y_train, k, f)
scores

[0.7368421052631579,
 0.9210526315789473,
 0.9736842105263158,
 0.9736842105263158,
 0.9736842105263158,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [57]:
# Treinando o modelo com as k=10 melhores características
clf.fit(Xz_train[:, indexes], y_train)
print('Acurácia treinamento', accuracy_score(clf.predict(Xz_train[:, indexes]), y_train)*100, '%')
print('Acurácia teste', accuracy_score(clf.predict(Xz_test[:, indexes]), y_test)*100, '%')

Acurácia treinamento 98.67549668874173 %
Acurácia teste 100.0 %
